# Decision Tree


For this problem, you will be implementing a Decision Tree classifier that works on discrete (categorical) features. Although a relatively simple learning algorithm, the Decision Tree is often used as a fundamental building block for more powerful (and popular) models such as Random Forest and Gradient Boosted ensembles. 

You should base your solution on the [ID3](https://en.wikipedia.org/wiki/ID3_algorithmhttps://en.wikipedia.org/wiki/ID3_algorithm) algorithm. This is a basic tree-learning algorithm that greedly grows a tree based on _information gain_ (reduction in entropy). Please refer to Chapter 3 of _Machine Learning_ by Tom M. Mitchell for more details. 


We have provided some skeleton code for the classifier, along with a couple of utility functions in the [decision_tree.py](./decision_tree.py) module. Please fill out the functions marked with `TODO` and feel free to add extra constructor arguments as you see fit (just make sure the default constructor solves the first dataset).


In [59]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We begin by loading necessary packages. Below follows a short description of the imported modules:

- `numpy` is the defacto python package for numerical calculation. Most other numerical libraries (including pandas) is based on numpy.
- `pandas` is a widely used package for manipulating (mostly) tabular data
- `decision_tree` refers to the module in this folder that should be further implemented by you

Note: The `%autoreload` statement is an [IPython magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) that automatically reloads the newest version of all imported modules within the cell. This means that you can edit the `decision_tree.py` file and just rerun this cell to get the updated version.

In [60]:
%autoreload

import numpy as np 
import pandas as pd 
import decision_tree as dt  # <-- Your implementation

## [1] First Dataset

The first dataset is a toy problem lifted from Table 3.2 in the Machine Learning textbook. The objective is to predict whether a given day is suitable for playing tennis based on several weather conditions. 

### [1.1] Load Data

We begin by loading data from the .csv file located in the same folder as this notebook.

In [61]:
data_1 = pd.read_csv('data_1.csv')
data_1

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


# Categorize data
1. Pick first column
2. Find unique values in column
3. Append each df from each value to a partition_array

In [62]:
columns = data_1.columns
all_partitions = []

for col in columns:
    col_partition = []
    uniqe_attr = data_1[col].unique()
    
    for attr in uniqe_attr:
        col_partition.append(data_1.loc[data_1[col] == attr])
        
    all_partitions.append(col_partition)
    
#all_partitions[3][0].loc[all_partitions["Temparature"] == "Sunny"]
all_partitions[0][0].loc[all_partitions[0][0]["Temperature"] == "Hot"]

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No


### [1.2] Fit and Evaluate Model

Next we fit and evaluate a Decision Tree over the dataset. We first partition the data into the dependent (`y` = Play Tennis) and independent (`X` = everything else) variables. We then initialize a Decision Tree learner and fit it to all the data. Finally, we evaluate the model over the same data by calculating its accuracy, i.e. the fraction of correctly classified samples.

Note that `.fit` and `.predict` will crash until you implement these two methods in [decision_tree.py](./decision_tree.py).

Assuming that you've correctly implemented the ID3 algorithm as described in the course textbook, you should expect the model to perfectly fit the training data. That is, you should get a classification accuracy of 100%.

In [63]:
# Separate independent (X) and dependent (y) variables
X = data_1.drop(columns=['Play Tennis'])
y = data_1['Play Tennis']

# Create and fit a Decrision Tree classifier
model_1 = dt.DecisionTree()  # <-- Should work with default constructor
model_1.fit(X, y)

# Verify that it perfectly fits the training set
print(f'Accuracy: {dt.accuracy(y_true=y, y_pred=model_1.predict(X)) * 100 :.1f}%')

0     Sunny
1     Sunny
7     Sunny
8     Sunny
10    Sunny
Name: Outlook, dtype: object
2     Overcast
6     Overcast
11    Overcast
12    Overcast
Name: Outlook, dtype: object
3     Rain
4     Rain
5     Rain
9     Rain
13    Rain
Name: Outlook, dtype: object
[4 5 5]
0     Hot
1     Hot
2     Hot
12    Hot
Name: Temperature, dtype: object
3     Mild
7     Mild
9     Mild
10    Mild
11    Mild
13    Mild
Name: Temperature, dtype: object
4    Cool
5    Cool
6    Cool
8    Cool
Name: Temperature, dtype: object
[4 4 6]
0     High
1     High
2     High
3     High
7     High
11    High
13    High
Name: Humidity, dtype: object
4     Normal
5     Normal
6     Normal
8     Normal
9     Normal
10    Normal
12    Normal
Name: Humidity, dtype: object
[7 7]
0     Weak
2     Weak
3     Weak
4     Weak
7     Weak
8     Weak
9     Weak
12    Weak
Name: Wind, dtype: object
1     Strong
5     Strong
6     Strong
10    Strong
11    Strong
13    Strong
Name: Wind, dtype: object
[6 8]


NotImplementedError: 

### [1.3] Inspect Classification Rules

A big advantage of Decision Trees is that they are relatively transparent learners. By this we mean that it is easy for an outside observer to analyse and understand how the model makes its decisions. The problem of being able to reason about how a machine learning model reasons is known as _Explainable AI_ and is often a desirable property of machine learning systems.

Every time a Decision Tree is evaluated, the datapoint is compared against a set of nodes starting at the root of the tree and (typically) ending at one of the leaf nodes. An equivalent way to view this reasoning is as an implication rule ($A \rightarrow B$) where the antecedent ($A$) is a conjunction of of attribute values and the consequent ($B$) is the predicted label. For instance, if a path down the tree first checks if Outlook=Rain, then checks if Wind=Strong, and then predicts Play Tennis=No, this line of reasoning can be represented as:

- If $Outlook=Rain \cap Wind=Strong \rightarrow$ then predict $Play Tennis = No$

We will leverage this property to export the decision tree you just created as a set of rules. For the subsequent cell to work, you must also have implemented the `.get_rules()` method in the provided boilerplate code.

In [ ]:
for rules, label in model_1.get_rules():
    conjunction = ' ∩ '.join(f'{attr}={value}' for attr, value in rules)
    print(f'{"✅" if label == "Yes" else "❌"} {conjunction} => {label}')

## [2] Second Dataset

The second dataset involves predicting whether an investment opportunity will result in a successful `Outcome` or not. To make this prediction, you are given a dataset of 200 historical$^1$ business ventures and their outcome, along with the following observed features:

- Whether the business oportunity is in a lucurative market or not 
- Whether the presented business idea has a competitive advantage
- Whether the second opinion from another investor is positive or not 
- The founder's previous experience with startups
- The founder's [Zodiac Sign](https://en.wikipedia.org/wiki/Astrology)

---
[1] Disclaimer: The dataset is not based on real-world business ventures. It is synthetic and generated by us. Also, it should not be considered financial advice.

### [2.1] Load Data

This dataset can also be found in a .csv file in the same folder as this notebook.

In [ ]:
data_2 = pd.read_csv('data_2.csv')
data_2

### [2.2] Split Data

We've also taken the liberty to pre-split the dataset into three different sets:

- `train` contains 50 samples that you should use to generate the tree
- `valid` contains 50 samples that you can use to evaluate different preprocessing methods and variations to the tree-learning algorithm.
- `test` contains 100 samples and should only be used to evaluate the final model once you're done experimenting.

In [ ]:
data_2_train = data_2.query('Split == "train"')
data_2_valid = data_2.query('Split == "valid"')
data_2_test = data_2.query('Split == "test"')
X_train, y_train = data_2_train.drop(columns=['Outcome', 'Split']), data_2_train.Outcome
X_valid, y_valid = data_2_valid.drop(columns=['Outcome', 'Split']), data_2_valid.Outcome
X_test, y_test = data_2_test.drop(columns=['Outcome', 'Split']), data_2_test.Outcome
data_2.Split.value_counts()

### [2.3] Fit and Evaluate Model

You may notice that the basic ID3 algorithm you developed for the first dataset does not generalize well when applied straight to this problem. Feel free to add extra functionality to it and/or the data preprocessing pipeline that might improve performance on the validation (and ultimately test set). As a debugging reference; it is highly possible to obtain accuracies over the validation and test set ranging from mid ~80% to low ~90%.

In [ ]:
# Fit model (TO TRAIN SET ONLY)
model_2 = dt.DecisionTree()  # <-- Feel free to add hyperparameters 
model_2.fit(X_train, y_train)

print(f'Train: {dt.accuracy(y_train, model_2.predict(X_train)) * 100 :.1f}%')
print(f'Valid: {dt.accuracy(y_valid, model_2.predict(X_valid)) * 100 :.1f}%')

## [3] Further steps (optional)

If you're done with the assignment but want to some more challenges; consider the following:

- Make a Decision Tree learner that can handle numerical attributes
- Make a Decision Tree learner that can handle numerical targets (regresion tree)
- Try implementing [Random Forest](https://en.wikipedia.org/wiki/Random_forest) on top of your Decision Tree algorithm

If you need more data for experimenting, UC Irvine hosts a [large repository](https://archive.ics.uci.edu/ml/datasets.php) of machine learning datasets.
